# Assignment 4.1 Supervised Learning

Nama : Muhammad Arsyad \
SCID : AI05113

akan memprediksi churn status in banking. berdasarkan data

In [ ]:
# import library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [ ]:
# load data
data = pd.read_csv('https://raw.githubusercontent.com/Rietaros/kampus_merdeka/main/SC_HW1_bank_data.csv')
data.head()

,RowNumber,CustomerId,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Geography        10000 non-null  object 
 3   Gender           10000 non-null  object 
 4   Age              10000 non-null  int64  
 5   Tenure           10000 non-null  int64  
 6   Balance          10000 non-null  float64
 7   NumOfProducts    10000 non-null  int64  
 8   HasCrCard        10000 non-null  int64  
 9   IsActiveMember   10000 non-null  int64  
 10  EstimatedSalary  10000 non-null  float64
 11  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 937.6+ KB


In [ ]:
data.columns

Index(['RowNumber', 'CustomerId', 'Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

In [ ]:
# hapus feature RowNumber, CustomerId
data.drop(['RowNumber', 'CustomerId'], axis=1, inplace=True)
data.columns

Index(['Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [ ]:
data[data.duplicated()].shape

(0, 10)

In [ ]:
data.isna().sum()

Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [ ]:
data_dummies = pd.get_dummies(data, drop_first=True)
data_dummies.head()

,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [ ]:
X = data_dummies.drop('Exited', axis=1)
y = data_dummies['Exited']

In [ ]:
scaler = MinMaxScaler()

X_minmax = scaler.fit_transform(X)
X_minmax

array([[0.32432432, 0.2       , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31081081, 0.1       , 0.33403148, ..., 0.        , 1.        ,
        0.        ],
       [0.32432432, 0.8       , 0.63635718, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.24324324, 0.7       , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.32432432, 0.3       , 0.29922631, ..., 1.        , 0.        ,
        1.        ],
       [0.13513514, 0.4       , 0.51870777, ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_minmax)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=8)

## Model 1. K Nearest Neighbors

pada medel pertama ini, saya akan menggunakan algoritma K Nearest Neighbors.
Cara Kerja K Nearest Neighbors adalah dengan mengklasifikan berdasarkan kedekatan atau kemiripan antar.
pada model ini saya akan mencari parameter terbaik untuk knn dengan menguji beberapa parameter yaitu `n_neighbors` saya ambil antara 1 hingga 10, `metric` saya gunakan minkowski, euclidean, manhattan.

In [ ]:
model_knn = KNeighborsClassifier()
params_knn = {'n_neighbors':list(range(1,10)),
         'metric':['minkowski', 'euclidean', 'manhattan']}
grid = GridSearchCV(estimator=model_knn,
                    param_grid=params_knn,
                    scoring='accuracy',
                    n_jobs = 10,
                    cv = 10)
grid.fit(X_train, y_train)
grid.best_params_

{'metric': 'minkowski', 'n_neighbors': 6}

diperoleh bahwa parameter terbaik adalah menggunakan `metric` minkowski dengan `n_neighbors` sebanyak 6

In [ ]:
y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.97      0.91      2405
           1       0.71      0.33      0.45       595

    accuracy                           0.84      3000
   macro avg       0.78      0.65      0.68      3000
weighted avg       0.83      0.84      0.82      3000


[[2326   79]
 [ 400  195]]

0.8403333333333334


diperoleh akurasi untuk model knn adalah 0.84 atau 84%

## Model 2. Decision Tree

pada model kedua ini, saya menggunakan algoritma decision tree. Decision tree mengambil keputusan berdasarkan serangkaian aturan. pada model ini saya akan mencari parameter terbaik dari parameter yang saya uji yaitu `criterion` : gini, entropy, dan log loss. `splitter` : best, random. dan `max_depth` yang saya gunakan adalah `None`, 50 dan 100

In [ ]:
model_tree = DecisionTreeClassifier(random_state=8)
params_tree = {'criterion':['gini', 'entropy', 'log_loss'],
               'splitter':['best', 'random'],
               'max_depth':[None, 50, 100]}
grid = GridSearchCV(estimator=model_tree,
                    param_grid=params_tree,
                    scoring='accuracy',
                    n_jobs = 10,
                    cv = 10)
grid.fit(X_train, y_train)
grid.best_params_

{'criterion': 'entropy', 'max_depth': None, 'splitter': 'best'}

diperoleh parameter terbaik yaitu {'criterion': 'entropy', 'max_depth': None, 'splitter': 'best'}

In [ ]:
y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.87      0.88      2405
           1       0.52      0.54      0.53       595

    accuracy                           0.81      3000
   macro avg       0.70      0.71      0.70      3000
weighted avg       0.81      0.81      0.81      3000


[[2103  302]
 [ 274  321]]

0.808


diperoleh akurasi untuk model decision tree adalah 0.80 atau 80%

## Model 3. Random Forest

lalu untuk model 3 saya menggunakan algoritma random forest, random forest mengambil keputusan berdasarkan keputusan dari beberapa tree (decision tree).
saya menguji dengan beberapa parameter seperti `n_etimators` : 50, 100, 150, 200. `criterion` : gini, entropy, log loss. dan `max_depth` : `None`, 50, 100

In [ ]:
model_forest = RandomForestClassifier(random_state=8)
params_forest = {'n_estimators':[50, 100, 150, 200],
                 'criterion':['gini', 'entropy', 'log_loss'],
                 'max_depth':[None, 50, 100]}
grid = GridSearchCV(estimator=model_forest,
                    param_grid=params_forest,
                    scoring='accuracy',
                    n_jobs = 10,
                    cv = 10)
grid.fit(X_train, y_train)
grid.best_params_

{'criterion': 'gini', 'max_depth': None, 'n_estimators': 150}

diperoleh bahwa parameter terbaik adalah {'criterion': 'gini', 'max_depth': None, 'n_estimators': 150}

In [ ]:
y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      2405
           1       0.74      0.49      0.59       595

    accuracy                           0.87      3000
   macro avg       0.81      0.73      0.76      3000
weighted avg       0.86      0.87      0.85      3000


[[2304  101]
 [ 302  293]]

0.8656666666666667


diperoleh akurasi untuk model 3 ini adalah 0.86 atau 86%

## Kesimpulan

dari ketiga algoritma yang telah saya uji, dan paramternya. saya mendapati kesimpulan bahwa algoritma atau model terbaik adalah random forest dengan parameter {'criterion': 'entropy', 'max_depth': 50, 'n_estimators': 200}.

menurut saya kenapa random forest menghasilkan akurasi tertinggi, karena kebanyakan data yang digunakan merupakan data diskrit. sehingga jika digunakan algoritma knn kurang baik. dan jika digunakan decision tree juga kurang baik karena hanya ada satu tree saja, sedangkan untuk random forest bisa dari beberapa tree.